In [1]:
# assumes you've downloaded data into ./data/source via ./download.sh

import os, glob, tables
import pandas as pd
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm

tqdm.pandas()

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
position_files = glob.glob('./data/source/*_positions.csv')

In [3]:
position_files

['./data/source/2019_05_28_positions.csv',
 './data/source/2019_05_11_positions.csv',
 './data/source/2019_05_06_positions.csv',
 './data/source/2019_05_02_positions.csv',
 './data/source/2019_05_15_positions.csv',
 './data/source/2019_05_05_positions.csv',
 './data/source/2019_05_12_positions.csv',
 './data/source/2019_05_16_positions.csv',
 './data/source/2019_05_01_positions.csv',
 './data/source/2019_05_25_positions.csv',
 './data/source/2019_05_21_positions.csv',
 './data/source/2019_05_18_positions.csv',
 './data/source/2019_05_26_positions.csv',
 './data/source/2019_05_31_positions.csv',
 './data/source/2019_05_08_positions.csv',
 './data/source/2019_05_22_positions.csv',
 './data/source/2019_05_23_positions.csv',
 './data/source/2019_05_09_positions.csv',
 './data/source/2019_05_30_positions.csv',
 './data/source/2019_05_27_positions.csv',
 './data/source/2019_05_19_positions.csv',
 './data/source/2019_05_20_positions.csv',
 './data/source/2019_05_24_positions.csv',
 './data/so

In [4]:
dfs = [pd.read_csv(f) for f in tqdm_notebook(position_files)]
df = pd.concat(dfs)

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
df.iloc[0]

route_id                           100210
timestamp                      1559064326
trip_id                          43271154
vehicle_id                           4524
vehicle_lat                       47.5388
vehicle_long                     -122.291
time_utc              2019-05-28 17:25:26
time_pct        2019-05-28 10:25:26-07:00
Name: 0, dtype: object

In [19]:
df.dtypes

route_id          int64
timestamp         int64
trip_id           int64
vehicle_id        int64
vehicle_lat     float64
vehicle_long    float64
time_utc         object
time_pct         object
dtype: object

In [17]:
pd.to_datetime(df.iloc[:1]['timestamp'], unit='s')

0   2019-05-28 17:25:26
Name: timestamp, dtype: datetime64[ns]

In [18]:
pd.to_datetime(df.iloc[:1]['time_pct'])

0   2019-05-28 10:25:26-07:00
Name: time_pct, dtype: datetime64[ns, pytz.FixedOffset(-420)]

In [7]:
# NOTE: pd.to_datetime is converting time_pct back into utc
df['timestamp_utc'] = df['time_pct'].progress_apply(pd.to_datetime)

  3%|▎         | 320490/10951273 [01:05<34:01, 5207.36it/s] 

KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.tslibs.tzconversion.tz_convert_single'
Traceback (most recent call last):
  File "pandas/_libs/tslibs/tzconversion.pyx", line 519, in pandas._libs.tslibs.tzconversion._tz_convert_dst
  File "pandas/_libs/tslibs/timezones.pyx", line 220, in pandas._libs.tslibs.timezones.get_dst_info
  File "pandas/_libs/tslibs/timezones.pyx", line 155, in pandas._libs.tslibs.timezones.get_utcoffset
  File "/opt/conda/lib/python3.7/site-packages/pytz/__init__.py", line 401, in utcoffset
    def utcoffset(self, dt):
KeyboardInterrupt: 
  5%|▍         | 505747/10951273 [01:42<37:35, 4630.81it/s]

KeyboardInterrupt: 

In [7]:
# output cleared for notebook (just cleared error output), but run this anyway
df.set_index('timestamp_utc', inplace=True)

In [8]:
df.sort_index(inplace=True)

In [9]:
df.head()

,route_id,timestamp,trip_id,vehicle_id,vehicle_lat,vehicle_long,time_utc,time_pct
timestamp_utc,,,,,,,,
2019-04-30 16:55:20-07:00,100264,1556668520,43273165,4542,47.602890,-122.331520,2019-04-30 23:55:20,2019-04-30 16:55:20-07:00
2019-04-30 16:55:20-07:00,100002,1556668520,43271314,4372,47.632500,-122.312492,2019-04-30 23:55:20,2019-04-30 16:55:20-07:00
2019-04-30 16:55:27-07:00,100133,1556668527,43258478,3430,47.480568,-122.208710,2019-04-30 23:55:27,2019-04-30 16:55:27-07:00
2019-04-30 16:55:29-07:00,100019,1556668529,43276252,2803,47.470131,-122.337158,2019-04-30 23:55:29,2019-04-30 16:55:29-07:00
2019-04-30 16:55:35-07:00,100187,1556668535,43137941,6847,47.763859,-122.193634,2019-04-30 23:55:35,2019-04-30 16:55:35-07:00


In [10]:
df.to_hdf('./data/intermed/positions_20190501.h5', key='df', mode='w')